In [55]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import glob

GIsis = nx.from_edgelist([[1,5],[5,3],[1,2],[2,6],[6,4],[6,7]])
#GIsis = nx.from_edgelist([[0, 1], [0, 2], [1, 10], [2, 9], [10, 7], [9, 8], [3, 6], [4, 6], [6, 7], [5, 8]])
sonda = []
sondaTemp = []

overhead_DataPlane_MPINT = []
overhead_ControlPlane_MPINT = []
overhead_DataPlane_MPolkaCRC8 = []
overhead_ControlPlane_MPolkaCRC8 = []
overhead_DataPlane_MPolkaCRC16 = []
overhead_ControlPlane_MPolkaCRC16 = []
overhead_DataPlane_INTClassico = []
overhead_ControlPlane_INTClassico = []
overhead_MPINT_Temp = []
overhead_MPolkaCRC8_Temp = []
overhead_MPolkaCRC16_Temp = []
overhead_INTClassico_Temp = []


def EdgeListToMST(path):
    edgeList = readTopology(path)
    #print(edgeList)
    G = nx.from_edgelist(edgeList)
    T = nx.minimum_spanning_tree(G,algorithm='prim')
    #print(T)
    return T

def GMLToMST(path):
    G = nx.read_gml(path,destringizer=int,label='id') #
    topologyName = G.graph['Network'] #Hooka o atributo network que identifica o nome da Topologia
    edgeList = list(map(list,G.edges()))
    #print(edgeList)
    G = nx.from_edgelist(edgeList)
    T = nx.minimum_spanning_tree(G,algorithm='prim')
    edgeList = list(map(list, T.edges()))
    T = nx.from_edgelist(edgeList)
    #print(T)
    return T,topologyName


def toDataframe(df,topology,sonda):
    #Overhead por abordagem
    overheadDP_mPolka = 0
    overheadDP_mPINT = 0
    overheadDP_INTClassico = 0
    overheadCP_mPolka = 0
    overheadCP_mPINT = 0
    overheadCP_INTClassico = 0

    matrixOverhead = {}
    matrixOverhead['Topology'] = [topology,topology]
    matrixOverhead['Where'] = ['DataPlane','ControlPlane']
    matrixOverhead['MPolka'] = [overheadDP_mPolka,overheadCP_mPolka]
    matrixOverhead['MPINT'] = [overheadDP_mPINT,overheadCP_mPINT]
    matrixOverhead['INT Clássico'] = [overheadDP_INTClassico,overheadCP_INTClassico]
    df2 = pd.DataFrame(data=matrixOverhead)
    result = pd.concat([df,df2],ignore_index=True)
    return result

def appendGraphToDataFrame(df,path):
    G,topologyName = GMLToMST(path)
    print(topologyName,'Edges = ', G.edges())
    #sonda.clear()
    sondaAux = dfs_init(G,node)
    print('Matriz de Sondas',topologyName,' =',sondaAux)
    print()
    df = toDataframe(df,topologyName,sondaAux)
    return df

def appendAllTopologysToDataFrame(df):
    listTopology = glob.glob('topologyZoo/*.gml')
    for topology in listTopology:
        df = appendGraphToDataFrame(df,topology)
    return df

def readTopology(path):
    edgeList = []

    with open(path,'r') as arq:
        lines = arq.readlines()
    
    for i in range(len(lines)):
        line = lines[i].split(' ')
        line.pop()
        line = list(map(int,line))
        edgeList.append(line)
        #print(line)
    
    return edgeList

def calculaOverheads(sonda,hops):
    #CABEÇALHO DO MPINT
    mPINT_Ethernet = 14
    mPINT_Ip = 20
    mPINT_INTHeader = 12
    mPINT_StackINT = 48
    mPINT_PathHeader = 1
    mPINT_StackPath = 4
    fixo_MPINT = mPINT_Ethernet + mPINT_Ip + mPINT_INTHeader + mPINT_PathHeader


    tamanhoSonda = len(sonda)
    if(hops == 0):
        return 0#SEM SALTOS, SEM OVERHEAD
    
    #PARA O MPINT
    overhead_MPINT_Temp.append(calculaOverheadMPINT(tamanhoSonda,hops))

    #PARA O MPOLKA CRC8

    #PARA O MPOLKA CRC16

def calculaOverheadMPINT(tamanhoSonda,hops):
    #CABEÇALHO DO MPINT
    mPINT_Ethernet = 14
    mPINT_Ip = 20
    mPINT_INTHeader = 12
    mPINT_StackINT = 48
    mPINT_PathHeader = 1
    mPINT_StackPath = 4
    fixo_MPINT = mPINT_Ethernet + mPINT_Ip + mPINT_INTHeader + mPINT_PathHeader

    if(hops == 0):
        return 0 #SEM SALTOS, SEM OVERHEAD

    #PARA O MPINT
    routeOverheadMPINT = mPINT_StackPath * hops
    telemetryOverheadMPINT = mPINT_StackINT * (tamanhoSonda-1)
    overhead = fixo_MPINT + routeOverheadMPINT + telemetryOverheadMPINT
    return overhead


def deadEndRelease(sonda,hops):
    #RELEASE OVERHEAD MPINT
    overhead_DataPlane_MPINT.append(overhead_MPINT_Temp.copy())
    overhead_ControlPlane_MPINT.append(calculaOverheadMPINT(len(sondaTemp)+1,hops))
    overhead_MPINT_Temp.clear()
    #RELEASE OVERHEAD MPOLKA

    #RELEASE OVERHEAD INT CLASSICO

    #RELEASE PROBE
    sonda.append(sondaTemp.copy())
    sondaTemp.clear()

def dfs(G,v,visited,hops,previousHop): #Retorna a matriz de sondas do MPolka
    visited[v] = True
    sondaTemp.append(v)
    grau = G.degree(v)
    calculaOverheads(sondaTemp,hops)
    hops += 1
    for w in G.neighbors(v):
        if grau == 1: #DEADEND
            deadEndRelease(sonda,hops)
        if not visited[w]:
            if grau == 2: #TRANSMISSÃO
                dfs(G,w,visited,hops,previousHop)
            else: #BIFURCAÇÃO
                previousHop = hops
                dfs(G,w,visited,hops,previousHop)
    return sonda

def dfs_init(G,v):
    visited=[False] * (G.number_of_nodes() + 1)
    sonda.clear()
    overhead_DataPlane_MPINT.clear()
    overhead_ControlPlane_MPINT.clear()
    overhead_DataPlane_MPolkaCRC8.clear()
    overhead_ControlPlane_MPolkaCRC8.clear()
    overhead_DataPlane_MPolkaCRC16.clear()
    overhead_ControlPlane_MPolkaCRC16.clear()
    overhead_ControlPlane_INTClassico.clear()
    overhead_DataPlane_INTClassico.clear()
    sondaAux = dfs(G,v,visited,hops=0,previousHop=0)
    return sondaAux

print('Digite o nó que enviará a sonda em G: ')
node = int(input())

df = pd.DataFrame(columns=['Topology','Where','MPolka','MPINT','INT Clássico']) #Empty Row Dataframe

sondaIsis = dfs_init(GIsis,node)
print('Matriz de Sondas Isis = ',sondaIsis,'\n')
df = toDataframe(df,'Isis',sondaIsis) #DataFrame Isis Example


print('DP MPINT:',overhead_DataPlane_MPINT)
print('CP MPINT:',overhead_ControlPlane_MPINT)

#df = appendAllTopologysToDataFrame(df)
#print(df)
#df.to_csv('overhead.csv',index=False)

#sns.displot(df, x="MPolka",hue="Where",element="step");
#sns.catplot(x="Topology", y="MPolka", hue="Where",kind="bar", data=df, errorbar = None);
#sns.catplot(x="Topology", y="MPINT", hue="Where",kind="bar", data=df, errorbar = None);
#sns.catplot(x="Topology", y="INT Clássico", hue="Where",kind="bar", data=df, errorbar = None);


Digite o nó que enviará a sonda em G: 
Matriz de Sondas Isis =  [[1, 5, 3], [2, 6, 4], [7]] 

DP MPINT: [[99, 151], [51, 103, 155], [59]]
CP MPINT: [203, 207, 111]
